In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#버전 2.0으로 업그레이드됨에 따라 v1과 호환이 되지 않음

In [108]:
from google.colab import files
upload = files.upload()

Saving data-01-test-score.csv to data-01-test-score.csv


In [0]:
filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue'
)
#batch가 shuffle되기 원하면 suffle_batch 이용
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

In [0]:
record_defaults = [[0.],[0.],[0.],[0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

In [0]:
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)
                    #x가 무엇인지, y가 무엇인지, 펌프마다 몇개를 가져올 것인지

In [0]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])
#                           3개가 x가 들어와서 한개의 y값이 나간다.
W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')
#                               y 하나
# Hypothesis
hypothesis = tf.matmul(X, W) + b


In [0]:
# cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [0]:
# Minimize. Need a very small learning rate for this data set
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5) # 0.0001
train = optimizer.minimize(cost)

In [136]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

coord =  tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord = coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run( [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

coord.request_stop()
coord.join(threads)

0 Cost:  36651.164 
Prediction:
 [[-38.150223]
 [-38.505978]
 [-41.955803]
 [-43.977325]
 [-28.761082]
 [-17.311611]
 [-32.130093]
 [-23.024555]
 [-31.984161]
 [-24.912163]]
10 Cost:  13.8767185 
Prediction:
 [[148.86494 ]
 [186.28046 ]
 [179.5237  ]
 [197.23454 ]
 [142.67485 ]
 [109.088455]
 [148.428   ]
 [110.833496]
 [178.10217 ]
 [169.6885  ]]
20 Cost:  18.340258 
Prediction:
 [[150.13953 ]
 [187.78802 ]
 [181.02196 ]
 [198.86282 ]
 [143.82138 ]
 [109.925575]
 [149.64854 ]
 [111.74145 ]
 [179.50124 ]
 [170.97458 ]]
30 Cost:  18.283815 
Prediction:
 [[150.16063]
 [187.78885]
 [181.0357 ]
 [198.87431]
 [143.81879]
 [109.91543]
 [149.65886]
 [111.75222]
 [179.49208]
 [170.95622]]
40 Cost:  18.18639 
Prediction:
 [[150.17323]
 [187.77959]
 [181.03946]
 [198.87494]
 [143.8085 ]
 [109.89964]
 [149.66104]
 [111.75694]
 [179.47353]
 [170.9292 ]]
50 Cost:  18.089312 
Prediction:
 [[150.18573]
 [187.77025]
 [181.04309]
 [198.87547]
 [143.79816]
 [109.88387]
 [149.66313]
 [111.76163]
 [179.45

In [137]:
#학습된 모델을 바탕으로
#Ask my score
print("I :", sess.run(hypothesis, feed_dict={X: [[100,70,101]]}))
print("Other", sess.run(hypothesis, feed_dict={X: [[60,70,110], [90,100,80]]}))

I : [[193.94958]]
Other [[180.24986]
 [172.61784]]
